In [5]:
import numpy as np 
import pandas as pd 

from subprocess import check_output
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from sklearn.cross_validation import  train_test_split
import time 
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from numpy import newaxis
from geopy.geocoders import Nominatim

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'


from subprocess import check_output
print(check_output(["ls", "/Users/mistersharma/Desktop"]).decode("utf8"))



/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


AIESEC
Booking Confirmation.pdf
CMP(orig)
ECG Project Report Version_4_2ndOct.pdf
ESSAY_STUFF.pages
GMAT
GX010030.MP4
ISB_ Indian School of Business _ Payment Details.pdf
ISB_ Indian School of Business _ Print Form.pdf
ISB_STATEMENT_OF_AUTHORISATION.pdf
ISB_STATEMENT_OF_INTEGRITY.pdf
Minor_SpatialDataMining
New Doc 2019-02-01 12.31.04.pdf
New Doc 2019-02-12 19.52.12.pdf
PROJECT_SEM6.rar
RAJESH_SEEMA_TRAVEL_INSURANCE.pdf
SONGS_ENGAGEMENT
Screen Shot 2019-01-11 at 8.31.37 PM.png
Screen Shot 2019-03-22 at 7.24.42 PM.png
Screen Shot 2019-03-23 at 5.35.58 PM.png
WhatsApp Image 2019-02-13 at 6.31.42 PM.jpeg
ZS_CAMPUS_BEATS_CHALLENGE
a.out
college_stuff
compilerSynopsis_new.pdf
database.csv
delhi peace marathon
randomphotos
umf13d_073.jpg
untitled folder
untitled folder 3



In [6]:
eq_dataset =  pd.read_csv("/Users/mistersharma/Desktop/database.csv", header=0)
eq_dataset.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,131.6,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,80.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,20.0,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic


In [7]:
print(len(eq_dataset[["Latitude","Longitude","Magnitude"]]))

23412


In [8]:
eq_dataset[["Latitude","Longitude","Magnitude"]][1:2]

,Latitude,Longitude,Magnitude
1,1.863,127.352,5.8


In [9]:
final_dataset = eq_dataset[["Latitude","Longitude","Magnitude"]]

In [10]:
np.array(final_dataset[0:1])

array([[ 19.246, 145.616,   6.   ]])

In [11]:
train_size = int(len(final_dataset) * 0.80)
test_size = len(final_dataset) - train_size
train, test = final_dataset[0:train_size], final_dataset[train_size:len(final_dataset)]
print(len(train), len(test))

18729 4683


In [12]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
    
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back)]
		dataX.append(np.array(a))
		dataY.append(np.array(dataset[i + look_back:i+look_back+1]))
        
	return np.array(dataX),np.array(dataY)

In [13]:
look_back = 50
trainX, trainY = create_dataset(train, look_back)

In [14]:
print(trainX.shape)
print(trainY.shape)

(18678, 50, 3)
(18678, 1, 3)


In [15]:
model = Sequential()
model.reset_states()

model.add(LSTM(
    input_dim=3,
    output_dim=100,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
    100,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(Dense(
    output_dim=3))
model.add(Activation('linear'))

start = time.time()
model.compile(loss='mse', optimizer='adam')
print ('compilation time : ', time.time() - start)



compilation time :  0.03558611869812012


In [16]:
model.fit(
    trainX,
    trainY,
    batch_size=500,
    nb_epoch=1,
    validation_split=0.05)

Train on 17744 samples, validate on 934 samples
Epoch 1/1
17744/17744 [==============================] - 41s 2ms/step - loss: 5923.3967 - val_loss: 5685.8610


In [17]:
test.shape
testX, testY = create_dataset(test, look_back)



In [18]:
test1 = np.array(testX[0:1])
print(test1[0:1][0:1])

[[[ -22.925  -70.237    6.8  ]
  [ -22.788  -70.195    5.6  ]
  [ -23.005 -174.996    5.8  ]
  [  -2.312  -77.838    6.8  ]
  [ -22.84   -70.601    5.5  ]
  [ -23.045  -70.287    5.5  ]
  [ -23.059  -70.598    5.9  ]
  [ -22.643  -66.323    6.   ]
  [ -23.104  -70.55     5.7  ]
  [ -21.185 -178.752    6.3  ]
  [ -21.968  173.486    6.   ]
  [  43.542 -127.414    5.8  ]
  [ -22.791  -70.769    5.9  ]
  [  -6.907  155.672    6.   ]
  [ -29.987 -177.942    5.9  ]
  [ -22.917  -70.483    6.1  ]
  [ -17.824 -173.225    5.7  ]
  [ -24.81   -68.472    5.8  ]
  [  -5.757  147.098    6.8  ]
  [   4.743   95.06     5.8  ]
  [  -4.63   151.869    5.9  ]
  [ -17.924 -174.683    5.5  ]
  [ -17.85  -174.768    5.6  ]
  [ -53.27     9.426    5.5  ]
  [ -23.77   -69.012    5.6  ]
  [ -16.116 -172.621    5.6  ]
  [  -2.808  101.162    5.9  ]
  [  -8.292  118.37     6.5  ]
  [  -2.238  100.413    6.   ]
  [  -8.224  118.467    6.5  ]
  [  37.392  141.586    5.9  ]
  [  15.28   -93.363    5.7  ]
  [  18.

In [19]:
model.predict(test1)

array([[[ 0.24762242,  1.7794907 ,  0.5343713 ],
        [ 0.67538327,  4.8310385 ,  1.5300336 ],
        [ 1.281926  ,  8.40331   ,  2.7282987 ],
        [ 1.851844  , 11.790063  ,  3.9321537 ],
        [ 2.1467354 , 12.976338  ,  4.328166  ],
        [ 2.205378  , 13.218896  ,  4.397273  ],
        [ 2.2201407 , 13.269954  ,  4.412202  ],
        [ 2.2243326 , 13.281108  ,  4.415213  ],
        [ 2.226274  , 13.283981  ,  4.4147167 ],
        [ 2.2293093 , 13.276801  ,  4.4195876 ],
        [ 2.1803277 , 13.23174   ,  4.4611607 ],
        [ 2.2322233 , 13.296384  ,  4.423997  ],
        [ 2.2356057 , 13.297648  ,  4.417072  ],
        [ 2.1999793 , 13.245337  ,  4.4618273 ],
        [ 2.2372422 , 13.290797  ,  4.4193916 ],
        [ 2.2463226 , 13.295712  ,  4.423523  ],
        [ 2.2410321 , 13.298469  ,  4.4254284 ],
        [ 2.2457616 , 13.297677  ,  4.4247966 ],
        [ 2.197378  , 13.244098  ,  4.461168  ],
        [ 2.2044823 , 13.247169  ,  4.469046  ],
        [ 2.2034528 